In [2]:
import os
import torch
import pickle
import numpy as np
import torch.nn as nn
import matplotlib as mpl
from scipy import interpolate
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib import rcParams
from captum.attr import GradientShap
from scipy.signal import savgol_filter
from torch.utils.data import Dataset, DataLoader
from utils import *
from qs_vae import *
from utils_cleaning import *
from utils_attributions import *
import warnings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.seterr(divide='ignore', invalid='ignore');
warnings.filterwarnings(action='ignore', message='All-NaN slice encountered')
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

### Content  
In this notebook, we use a subset of 9 models that best represent the attributions from the entire swarm of 50 models on a random sample of spectra. Reducing the number of models to 9 allows sue to make the attribution calculation for the entire pre-flare dataset tractable. 

Using these models we calculate the average attention for every spectrum in the entire PF dataset with respect to the ***Expected Gradients*** formalism. This represents a fast implementation with reduced models and number of background references. The full version with background references set to 500 and models to 50 should be run on key examples, but for now the results should be adequate. The notebooks return three matrices per observation 
- ***y_hats*** $\to$ A dataset of shape (step, time, y) that contains the probability of each spectrum in the observation belonging to the PF-class  
- ***y_raws*** $\to$ A dataset of shape (step, time, y) that contains the raw output of the positive channel of the network before a Softmax activation function. The raw output allows us to absorb and reflect the high scores in the heatmaps color, which would otherwise be saturated at 1  
- ***attributions*** $\to$ A dataset of shape (step, time, y, lambda) that consists of attributions instead of spectra    

The data structure is a one-to-one of the input observation, meaning all the attribution data for a particular spectrum can be found at the same location as the spectrum in the original dataset 

***Select ensemble model subset to use and average over***  

In [3]:
fold = 44 # model corresponding to a particular test/train split of the observations
degree_of_smooth = 41 # post-hoc smooth of attributions
baseline_samples = 500 # number of background data used to calculate attributions

In [4]:
# collect paths to models
path_to_models = '/data1/userspace/bpanos/XAI/models/EG/'
model_paths = [ f'{path_to_models}{name}' for name in os.listdir(path_to_models) ]

***Construct background dataset***  
Attributions in Expected Gradients are calculated by comparing the output of the network when different backgrounds are joined to the original spectra with varying strengths 

In [6]:
with open(f'/data1/userspace/bpanos/XAI/models/ConvNet/{fold}.p', 'rb') as f: dic = pickle.load(f)
path_to_clean_data = '/data1/userspace/bpanos/XAI/data/ConvNet/clean_strong/'
train_obs = dic['train_obs']
AR_train = []
PF_train = []
for n in train_obs:
    if 'AR' in n:
        AR_train.append(n)
    if 'PF' in n:
        PF_train.append(n)
X_train_AR = None
X_train_PF = None
for AR_file, PF_file in zip(AR_train, PF_train):
    fhand = np.load(path_to_clean_data + AR_file)
    X_AR = fhand['X']
    try: X_train_AR = np.concatenate( (X_train_AR, X_AR), axis=0 )
    except: X_train_AR = X_AR
    fhand = np.load(path_to_clean_data + PF_file)
    X_PF = fhand['X']
    try: X_train_PF = np.concatenate( (X_train_PF, X_PF), axis=0 )
    except: X_train_PF = X_PF
X_train_AR, X_train_PF = Balance(X_train_AR, X_train_PF)
y_train_AR = np.zeros(len(X_train_AR))
y_train_PF = np.ones(len(X_train_PF))
X_train = np.concatenate((X_train_AR, X_train_PF), axis=0)
y_train = np.concatenate((y_train_AR, y_train_PF))
baseline = torch.Tensor(X_train).to(device).unsqueeze(1)

***Calculate the attributions of every spectra in every PF observation***

In [ ]:
# set path to data and path to save attributions
path_to_data = '/data1/iris_level_2C_10DNs/'
path_to_save = '/data1/userspace/bpanos/XAI/data/attributions/EG/'

# iterate over all PF observations
for file in os.listdir(path_to_data):
    if 'PF' not in file: continue # only compute attributions for PF obs
    if file in os.listdir(path_to_save): continue
    obs = path_to_data + file
    print(obs)
    # load data, these functions are more than required
    nprof_orig, data = process_obs(vae_model, obs, mode='Raw')
    _, nprof_clean = process_obs(vae_model, obs, mode='clean')
    
    # collect params of data shape
    n_y = data.shape[2]
    n_time = data.shape[1]
    n_steps = data.shape[0]
    n_lambda = data.shape[3]
    
    # initiate empty matrices to store the attributions and scores
    y_raw_cube = np.zeros(data.shape[:-1], dtype='float32')
    y_hat_cube = np.zeros(data.shape[:-1], dtype='float32')
    attribution_cube = np.zeros_like(data, dtype='float32')
    
    # iterate over every spectra in the observation and fill in the empty matrices
    for time in range(n_time):
        for step in range(n_steps):
            for y in range(n_y):
                
                spectrum = data[step, time, y, :]

                # attributions of single spectrum are calculated here
                y_raws = []
                y_hats = []
                attributions = []
                for path in model_paths:
                    model = CNN(num_classes=2)
                    model.load_state_dict(torch.load(path))
                    model.to(device)
                    model.eval();
                    y_raw, y_hat, attribution = EG_attributions(spectrum, baseline, model, n_samples=baseline_samples, degree_of_smooth=degree_of_smooth)
                    y_raws.append(y_raw)
                    y_hats.append(y_hat)
                    attributions.append(attribution)
                    del model

                # avarage over all models
                y_raw = np.nanmean(y_raws)
                y_raw_cube[step, time, y] = y_raw

                y_hat = np.nanmean(y_hats)
                y_hat_cube[step, time, y] = y_hat

                attributions = np.vstack(attributions)
                attributions = np.nanmean(attributions, axis=0)
                attribution_cube[step, time, y, :] = attributions
    
    # save matrices
    np.savez( path_to_save + file[:-4], y_raws=y_raw_cube, y_hats=y_hat_cube, attributions=attribution_cube)